In [10]:
import torch
from senreps import proc_embeddings, proc_embeddings_bertje
from torch import Tensor
from transformers import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

'''
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True).to(device=device)
task = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
'''
model = BertModel.from_pretrained('bert-base-dutch-cased', output_hidden_states=True).to(device=device)
tokenizer = BertTokenizer.from_pretrained('bert-base-dutch-cased')
task = BertForMaskedLM.from_pretrained('bert-base-dutch-cased').to(device)


In [4]:
from polyglot.mapping import Embedding
from polyglot.downloader import downloader
from nltk import word_tokenize

nl_en = {}

embeddings = Embedding.load("embeddings2/nl/embeddings_pkl.tar.bz2")

with open('data/filtered_en_nl_dict.txt', 'r') as bidict:
    for trans in bidict:
        nl, en = trans.split()
        nl_en[nl] = en

In [5]:
def preprocess_data(path):
    proc_data = []
    with open(path, 'r') as data: 
        for sentence in data:
            proc_sentence = word_tokenize(sentence.lower())
            proc_data.append(proc_sentence)
    
    return proc_data

In [6]:
train_data = preprocess_data('data/train.txt')
dev_data = preprocess_data('data/valid.txt')
test_data = preprocess_data('data/test.txt')
print(train_data[0:5])

[['biologie', 'is', 'de', 'natuurwetenschap', 'die', 'zich', 'richt', 'op', 'levende', 'organismen', ',', 'levensprocessen', 'en', 'levensverschijnselen', '.'], ['de', 'biologie', 'omvat', 'een', 'breed', 'scala', 'aan', 'vakgebieden', 'waarin', 'men', 'onderzoek', 'doet', 'naar', 'fysieke', 'structuur', ',', 'chemische', 'processen', ',', 'moleculaire', 'interacties', ',', 'fysiologische', 'mechanismen', ',', 'ecologische', 'dynamiek', ',', 'ontwikkeling', 'en', 'evolutie', '.'], ['biologie', 'erkent', 'de', 'cel', 'als', 'de', 'fysieke', 'basiseenheid', 'van', 'het', 'leven', ',', 'genen', 'als', 'de', 'basiseenheid', 'van', 'erfelijke', 'informatie', 'en', 'evolutie', 'als', 'de', 'drijvende', 'kracht', 'achter', 'het', 'ontstaan', 'en', 'het', 'uitsterven', 'van', 'soorten', '.'], ['levende', 'organismen', 'zijn', 'open', 'systemen', 'die', 'in', 'staat', 'zijn', 'te', 'overleven', 'door', 'bruikbare', 'omzettingen', 'van', 'energie', 'en', 'door', 'handhaving', 'van', 'hun', 'vita

In [11]:
data = {'train':[], 'dev':[], 'test':[]}

data['train'] = proc_embeddings_bertje(train_data, model, tokenizer, embeddings, nl_en, 'TF', device)
data['dev'] = proc_embeddings_bertje(dev_data, model, tokenizer, embeddings, nl_en, 'TF', device)
data['test'] = proc_embeddings_bertje(test_data, model, tokenizer, embeddings, nl_en, 'TF', device)

torch.save(data, 'xlingual_data_dutch.pt')


100%|██████████| 10000/10000 [03:37<00:00, 45.96it/s]


In [ ]:
from train import train

W = train(data, epochs=100, batch_size=64, lr=1e-4, early_stopping=True)

print(W)

In [ ]:
torch.save(W, 'proj_matrix.pt')

In [ ]:
input_ids = Tensor([tokenizer.encode('I want to [MASK] the bike because it is cheap')]).type(torch.long).to(device)
input_embs = model(input_ids)[-1][0]
mask_embs = model(Tensor([103]).unsqueeze(0).type(torch.long).to(device))[-1][0]
print(masked_labels.size())
output1 = task(inputs_embeds=input_embs)
output2 = task(input_ids)

print(output1.size())
print(output2)

In [ ]:
indices = Tensor([0, 2]).type(torch.long)
predicted_token = torch.argmax(output1[0][indices])

print(predicted_token)
